   ## Applying Functions to Data - Part 2

  ## Setup

  We are going to use some real data about higher education institutions from the college scorecard (https://collegescorecard.ed.gov/) to explore the types of conclusions we can make from the data. The college scorecard releases data on higher education institutions to help make the institutions more transparent and provide a place for parents, students, educators, etc can get information about specific institutions from a third party (i.e. US Department of Education).

  ### Loading R packages

In [ ]:
library(tidyverse)
library(ggformula)
library(mosaic)

theme_set(theme_bw(base_size = 16))

college_score <- read_csv("https://raw.githubusercontent.com/lebebr01/statthink/master/data-raw/College-scorecard-4143.csv", guess_max = 10000)
head(college_score)

   ## Considering Groups
   We've spent a lot of time trying to reason about other attributes that may be important in explaining variation in our attribute of interest. So far we have only explored the attribute without considering other attributes, in practice that is not that useful.

   Instead, it is common to compute conditional statistics based on other characteristics in the data. An example may help to show the idea more clearly.

In [ ]:
college_score %>%
  df_stats(adm_rate ~ region, median)

   Presented above are the conditional medians for the higher education institutions in different areas of the country. More specifically, the data are essentially split into subgroups and the median is computed for each of those subgroups instead of pooling all institutions into a single data frame. The formula syntax is now `outcome ~ grouping` where the attribute of interest (i.e. commonly a numeric attribute) and the attribute to the right of the `~` is the grouping attribute. This syntax is similar to the violin plots that were created earlier.

   Can you see differences in the admission rates across the regions?

   One thing that is useful to add in when computing conditional statisics, is how many data points are in each group. This is particularly useful when the groups are different sizes, which is common. To do this, we can add another function to the `df_stats()` function.

In [ ]:
college_score %>%
  df_stats(adm_rate ~ region, median, length)

   This adds another columns which represents the number of observations that went into the median calculation for each group. The syntax above also shows that you can add additional functions separated by a comma in the `df_stats()` function and are not limited to a single function. We will take advantage of this feature later on.

   ### Adding additional groups
   What if we thought more than one attribute was important in explaining variation in the outcome attribute? These can also be added to the `df_stats()` function for additional conditional statistics. The key is to add another attribute to the right-hand side of the formula argument. More than one attribute are separated with a `+` symbol.

In [ ]:
college_score %>%
  df_stats(adm_rate ~ region + preddeg, median, length)

   ## Other statistics of center
   So far we have been discussing the median. The median attempts to provide a single number summary for the center of the distribution. It is a robust statistic, but likely isn't the most popular statistic to provide a location for the center of a distribution. The mean is often more commonly used as a measure of the center of a distribution. Part of this is due to the usage of the mean in common statistical methods and the mean also uses the values of all the data in the calculation. The median only considers the values of the middle score or scores, therefore this statistic is less sensitive to extreme values than the mean. I like to look at both statistics and this can provide some insight into the distribution of interest. We can add the mean using the `df_stats()` function by adding the function `mean`.

In [ ]:
stats_compute <- college_score %>%
  df_stats(adm_rate ~ region, median, mean, length)
stats_compute

   Do you notice any trends in the direction the mean and median typically follow? More specifically, is the mean typically larger than the median or vice versa?

  Let's visualize them.

In [ ]:
gf_histogram(~ adm_rate, data = college_score, bins = 30) %>%
  gf_facet_wrap(~ region) %>%
  gf_vline(color = 'blue', xintercept = ~ median, data = stats_compute, size = 1) %>%
  gf_vline(color = 'lightblue', xintercept = ~ mean, data = stats_compute, size = 1)

   What is different about the distributions that have larger differences in the mean and median?

  ### Variation by Group
  These statistics can also be calculated by different grouping attributes similar to what was done with statisitcs of center. Now the attribute of interest is on the left-hand side of the equation and the grouping attribute is on the right hand side.

In [ ]:
iqr_groups <- college_score %>%
  df_stats(adm_rate ~ region, IQR, quantile(c(0.25, 0.75)), nice_names = TRUE)
iqr_groups

  This can also be visualized to see how these statistics vary across the groups.

In [ ]:
gf_histogram(~ adm_rate, data = college_score, bins = 30, color = 'black') %>%
  gf_vline(color = 'blue', xintercept = ~ value, 
           data = filter(pivot_longer(iqr_groups, cols = c(IQR, `X25.`, `X75.`)), name %in% c('X25.', 'X75.')), size = 1) %>%
  gf_facet_wrap(~ region)

  ## Other measures of variation
  There are many other variation measures that are used in statistics. We will apply a functional approach to these and try to visualize what they are trying to represent. The statistics discussed here represent deviations from the mean, either the average absolute deviation or the average squared deviation.

In [ ]:
college_score %>%
  df_stats(~ adm_rate, sd, var)

  In order to compute the mean absolute error, we first need to define a new function.

In [ ]:
mae <- function(x, na.rm = TRUE, ...) {
  avg <- mean(x, na.rm = na.rm, ...)
  abs_avg <- abs(x - avg)
  
  mean(abs_avg)
}

  We can now use this new function just like any other function.

In [ ]:
college_score %>%
  df_stats(~ adm_rate, sd, var, mae)